In [30]:
import findspark
findspark.init()

In [31]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.streaming import *
from pyspark.sql.types import *

spark = SparkSession.builder.master("local").appName("SortFilter").getOrCreate()
sc = spark.sparkContext

In [32]:
# sql adaptive query execution adaptive.coalescePartitions.enabled will makr paritions dynamic
sc.setLogLevel("Error")
spark.conf.set("spark.sql.shuffle.partitions",3)
spark.conf.get("spark.sql.shuffle.partitions")
spark.conf.set("spark.sql.adaptive.enabled","true")
spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled","true")

"""
Which one results faster
1.filter > sort > count
2.sort > filter > count
3.cache > filter > sort
4.cache > sort > filter
"""

filepath = "file:///C:/Users/venka/PycharmProjects/pythonProject/dataset/"

df1=spark.read.csv(filepath + "SortFilterSales.csv",header=True,inferSchema=True)
#.option('delimiter','~|') can't be more than one character error

df1.show(truncate=0)
df1.printSchema()

+---------------------------------+----------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+----------+-----------+-------------+------------------+---------------+
|Region                           |Country   |Item Type      |Sales Channel|Order Priority|Order Date|Order ID |Ship Date |Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost|Total Profit|Order year|Order Month|Order Weekday|Unit Margin       |Order_Ship_Days|
+---------------------------------+----------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+----------+-----------+-------------+------------------+---------------+
|Middle East and North Africa     |Libya     |Cosmetics      |Offline      |M             |2014-10-18|686800706|2014-10-31|8446      |437.2     |263.33   |3692591.2    |2224085.18|1468506.02  |2014     

In [33]:
%%time
df_meth1_filter = df1.filter('"Sales Channel" == "Online"')
df_meth1_sort = df_meth1_filter.sort("Order Date")
df_meth1_count = df_meth1_sort.count()
print(f"Sort records {df_meth1_count}")

df_meth1_sort.explain()

Sort records 0
== Physical Plan ==
LocalTableScan <empty>, [Region#3104, Country#3105, Item Type#3106, Sales Channel#3107, Order Priority#3108, Order Date#3109, Order ID#3110, Ship Date#3111, Units Sold#3112, Unit Price#3113, Unit Cost#3114, Total Revenue#3115, Total Cost#3116, Total Profit#3117, Order year#3118, Order Month#3119, Order Weekday#3120, Unit Margin#3121, Order_Ship_Days#3122]


CPU times: total: 0 ns
Wall time: 74.8 ms


In [34]:
%%time
df_meth2_sort = df1.sort("Order Date")
df_meth2_filter = df_meth2_sort.filter('"Sales Channel" == "Online"')
df_meth2_count = df_meth2_filter.count()
print(f"Sort records {df_meth2_count}")

df_meth2_filter.explain()

Sort records 0
== Physical Plan ==
LocalTableScan <empty>, [Region#3104, Country#3105, Item Type#3106, Sales Channel#3107, Order Priority#3108, Order Date#3109, Order ID#3110, Ship Date#3111, Units Sold#3112, Unit Price#3113, Unit Cost#3114, Total Revenue#3115, Total Cost#3116, Total Profit#3117, Order year#3118, Order Month#3119, Order Weekday#3120, Unit Margin#3121, Order_Ship_Days#3122]


CPU times: total: 0 ns
Wall time: 72.6 ms


In [35]:
%%time
df2=spark.read.csv(filepath + "SortFilterSales.csv",header=True,inferSchema=True)
df2.cache()
df_meth3_filter = df2.filter('"Sales Channel" == "Online"')
df_meth3_sort = df_meth3_filter.sort("Order Date")
df_meth3_count = df_meth3_sort.count()
print(f"Sort records {df_meth3_count}")

df_meth3_sort.explain()

Sort records 0
== Physical Plan ==
LocalTableScan <empty>, [Region#4444, Country#4445, Item Type#4446, Sales Channel#4447, Order Priority#4448, Order Date#4449, Order ID#4450, Ship Date#4451, Units Sold#4452, Unit Price#4453, Unit Cost#4454, Total Revenue#4455, Total Cost#4456, Total Profit#4457, Order year#4458, Order Month#4459, Order Weekday#4460, Unit Margin#4461, Order_Ship_Days#4462]


CPU times: total: 15.6 ms
Wall time: 320 ms


In [36]:
%%time
df3=spark.read.csv(filepath + "SortFilterSales.csv",header=True,inferSchema=True)
df3.cache()
df_meth4_sort = df3.sort("Order Date")
df_meth4_filter = df_meth4_sort.filter('"Sales Channel" == "Online"')
df_meth4_count = df_meth4_filter.count()
print(f"Sort records {df_meth4_count}")

df_meth4_filter.explain()

Sort records 0
== Physical Plan ==
LocalTableScan <empty>, [Region#4904, Country#4905, Item Type#4906, Sales Channel#4907, Order Priority#4908, Order Date#4909, Order ID#4910, Ship Date#4911, Units Sold#4912, Unit Price#4913, Unit Cost#4914, Total Revenue#4915, Total Cost#4916, Total Profit#4917, Order year#4918, Order Month#4919, Order Weekday#4920, Unit Margin#4921, Order_Ship_Days#4922]


CPU times: total: 0 ns
Wall time: 262 ms
